<a href="https://colab.research.google.com/github/BhanuPrakash-16/GAN_for_images_Project-4/blob/main/GAN_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Module-1 Global Parameter setting and Importing

# Imports

import tensorflow as tf
import time
from tensorflow.keras import layers, Model

import os
import numpy as np
import tensorflow_datasets as tfds
import subprocess
from scipy.signal import butter, filtfilt


!pip install wfdb # Install wfdb package
import wfdb


import csv
import logging
from datetime import datetime   # ✅ REQUIRED

import matplotlib.pyplot as plt

# ---------------- CONFIG ----------------


SEQ_LEN = 1248
LATENT_DIM = 100
BATCH_SIZE = 64
EPOCHS = 2
N_CRITIC = 5
LR = 1e-4
GP_LAMBDA = 10.0

# =========================================================
# CONFIG
# =========================================================
FS = 300        # Sampling rate for PhysioNet 2017
LOWCUT = 0.5
HIGHCUT = 40.0

In [ ]:
# Module 2 Data set loading and preprocessing

# =========================================================
# PREPROCESSING
# =========================================================

def bandpass_filter(signal, fs, low=0.5, high=40.0):
    """
    Band-pass filter for ECG (baseline wander + noise removal)
    """
    b, a = butter(
        N=4,
        Wn=[low / (fs / 2), high / (fs / 2)],
        btype="band"
    )
    return filtfilt(b, a, signal)


def normalize(signal):
    """
    Z-score normalization
    """
    return (signal - np.mean(signal)) / (np.std(signal) + 1e-8)


def preprocess_ecg_segment(segment, fs, target_len):
    """
    Preprocess ECG segment and ENFORCE fixed length
    (CRITICAL for GAN stability)
    """
    segment = bandpass_filter(segment, fs)
    segment = normalize(segment)

    # enforce exact length
    if len(segment) > target_len:
        segment = segment[:target_len]
    elif len(segment) < target_len:
        pad_len = target_len - len(segment)
        segment = np.pad(segment, (0, pad_len), mode="constant")

    return segment.astype("float32")

# =========================================================
# SEGMENTATION
# =========================================================

def segment_ecg(signal, seq_len):
    """
    Split long ECG signal into non-overlapping fixed windows
    """
    return [
        signal[i:i + seq_len]
        for i in range(0, len(signal) - seq_len + 1, seq_len)
    ]

# =========================================================
# PHYSIONET DIRECTORY LOADER
# =========================================================

def load_physionet_ecg(
    physionet_dir,
    seq_len=1250,
    lead_index=0
):
    """
    Load, preprocess, and segment PhysioNet ECG data
    from a local directory containing .hea/.dat files
    """

    if not os.path.isdir(physionet_dir):
        raise FileNotFoundError(
            f"PhysioNet directory not found: {physionet_dir}"
        )

    all_segments = []

    print(f"[INFO] Scanning PhysioNet directory: {physionet_dir}")

    for file in os.listdir(physionet_dir):
        if file.endswith(".hea"):
            record_name = file.replace(".hea", "")
            record_path = os.path.join(physionet_dir, record_name)

            # Load ECG record
            record = wfdb.rdrecord(record_path)
            fs = record.fs
            signal = record.p_signal[:, lead_index]

            # Segment signal
            segments = segment_ecg(signal, seq_len)

            for seg in segments:
                seg = preprocess_ecg_segment(
                    seg,
                    fs=fs,
                    target_len=seq_len
                )
                all_segments.append(seg)

    if not all_segments:
        raise RuntimeError(
            "No ECG segments extracted. "
            "Check PhysioNet directory contents."
        )

    ecg_array = np.array(all_segments)[..., np.newaxis]

    print(f"[INFO] Total ECG segments loaded : {ecg_array.shape[0]}")
    print(f"[INFO] Segment shape              : {ecg_array.shape[1:]}")

    # 🔐 Safety check
    assert ecg_array.shape[1] == seq_len, (
        f"Segment length mismatch: expected {seq_len}, "
        f"got {ecg_array.shape[1]}"
    )

    return ecg_array

# =========================================================
# TENSORFLOW DATASET CONVERSION
# =========================================================

def get_tf_dataset(ecg_array, batch_size, shuffle=True):
    """
    Convert NumPy ECG array to tf.data.Dataset
    """
    ds = tf.data.Dataset.from_tensor_slices(ecg_array)

    if shuffle:
        ds = ds.shuffle(buffer_size=1024)

    ds = ds.batch(batch_size, drop_remainder=True)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds


In [ ]:
# Module 3 Build Generator

def build_generator(seq_len, latent_dim):
    """
    Builds and returns the ECG Generator model
    (NO printing here)
    """

    z = layers.Input(shape=(latent_dim,), name="latent_vector")

    x = layers.Dense((seq_len // 8) * 128, name="dense_projection")(z)
    x = layers.Reshape((seq_len // 8, 128), name="reshape")(x)

    for i, filters in enumerate([128, 64, 32], start=1):
        x = layers.UpSampling1D(2, name=f"upsample_{i}")(x)
        x = layers.Conv1D(filters, 5, padding="same", name=f"conv1d_{i}")(x)
        x = layers.BatchNormalization(name=f"bn_{i}")(x)
        x = layers.ReLU(name=f"relu_{i}")(x)

    out = layers.Conv1D(
        1, 7, padding="same", activation="tanh", name="ecg_output"
    )(x)

    model = Model(z, out, name="ECG_Generator")
    return model

# Build model
generator = build_generator(SEQ_LEN, LATENT_DIM)

# PRINT ARCHITECTURE OUTSIDE THE FUNCTION

print("\n" + "=" * 50)
print("ECG GENERATOR ARCHITECTURE")
print("=" * 50)

generator.summary()

print("=" * 50 + "\n")



ECG GENERATOR ARCHITECTURE


Model: "ECG_Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ latent_vector (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_projection (Dense)        │ (None, 19968)          │     2,016,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 156, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ upsample_1 (UpSampling1D)       │ (None, 312, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 312, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_1 (BatchNormalization)       │ (None, 312, 128)       │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_1 (ReLU)                   │ (None, 312, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ upsample_2 (UpSampling1D)       │ (None, 624, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 624, 64)        │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_2 (BatchNormalization)       │ (None, 624, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_2 (ReLU)                   │ (None, 624, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ upsample_3 (UpSampling1D)       │ (None, 1248, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 1248, 32)       │        10,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_3 (BatchNormalization)       │ (None, 1248, 32)       │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_3 (ReLU)                   │ (None, 1248, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ecg_output (Conv1D)             │ (None, 1248, 1)        │           225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,151,233 (8.21 MB)

 Trainable params: 2,150,785 (8.20 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Module 4 Build Critic

def build_critic(seq_len):
    """
    Builds and returns the ECG Critic (WGAN-GP)
    """

    ecg_input = layers.Input(
        shape=(seq_len, 1),
        name="ecg_input"
    )

    x = ecg_input

    for i, filters in enumerate([32, 64, 128, 256], start=1):
        x = layers.Conv1D(
            filters=filters,
            kernel_size=5,
            strides=2,
            padding="same",
            name=f"conv1d_{i}"
        )(x)
        # ✅ FIX: use negative_slope instead of alpha
        x = layers.LeakyReLU(
            negative_slope=0.2,
            name=f"leaky_relu_{i}"
        )(x)

    x = layers.Flatten(name="flatten")(x)

    critic_output = layers.Dense(
        1,
        name="critic_score"
    )(x)y

    model = Model(
        inputs=ecg_input,
        outputs=critic_output,
        name="ECG_Critic"
    )

    return model


# ------------------ USAGE ------------------

critic = build_critic(SEQ_LEN)

# PRINT ARCHITECTURE OUTSIDE THE FUNCTION
print("\n" + "=" * 50)
print("ECG Critic ARCHITECTURE")
print("=" * 50)

critic.summary()

print("=" * 50 + "\n")



ECG Critic ARCHITECTURE


Model: "ECG_Critic"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ecg_input (InputLayer)          │ (None, 1248, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 624, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu_1 (LeakyReLU)        │ (None, 624, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 312, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu_2 (LeakyReLU)        │ (None, 312, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 156, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu_3 (LeakyReLU)        │ (None, 156, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 78, 256)        │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu_4 (LeakyReLU)        │ (None, 78, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 19968)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ critic_score (Dense)            │ (None, 1)              │        19,969 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,649 (920.50 KB)

 Trainable params: 235,649 (920.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Monitoring and controlling code

class WGANMonitor:
    """
    Monitoring and control module for WGAN training
    """

    def __init__(
        self,
        log_dir="logs",
        patience=10,
        min_delta=0.001
    ):
        os.makedirs(log_dir, exist_ok=True)

        self.writer = tf.summary.create_file_writer(log_dir)

        self.patience = patience
        self.min_delta = min_delta

        self.best_g_loss = np.inf
        self.wait = 0

        self.g_history = []
        self.c_history = []

        print(f"[Monitor] Logging to: {log_dir}")

    # -----------------------------------------------------
    # LOG LOSSES
    # -----------------------------------------------------
    def log(self, epoch, g_loss, c_loss):

        g_loss = float(g_loss)
        c_loss = float(c_loss)

        self.g_history.append(g_loss)
        self.c_history.append(c_loss)

        with self.writer.as_default():
            tf.summary.scalar("Generator_Loss", g_loss, step=epoch)
            tf.summary.scalar("Critic_Loss", c_loss, step=epoch)

        print(
            f"[Monitor] Epoch {epoch} | "
            f"G Loss: {g_loss:.4f} | "
            f"C Loss: {c_loss:.4f}"
        )

    # -----------------------------------------------------
    # CHECK FOR NaN
    # -----------------------------------------------------
    def check_nan(self, g_loss, c_loss):
        if np.isnan(g_loss) or np.isnan(c_loss):
            print("[Monitor] NaN detected! Stopping training.")
            return True
        return False

    # -----------------------------------------------------
    # EARLY STOPPING
    # -----------------------------------------------------
    def early_stopping(self, g_loss):
        if g_loss < self.best_g_loss - self.min_delta:
            self.best_g_loss = g_loss
            self.wait = 0
        else:
            self.wait += 1

        if self.wait >= self.patience:
            print("[Monitor] Early stopping triggered.")
            return True

        return False

    # -----------------------------------------------------
    # PLOT FINAL LOSS CURVES
    # -----------------------------------------------------
    def plot_losses(self, save_path=None):
        plt.figure(figsize=(8, 5))
        plt.plot(self.g_history, label="Generator Loss")
        plt.plot(self.c_history, label="Critic Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("WGAN-GP Training Loss Curves")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        if save_path:
            plt.savefig(save_path)

        plt.show()

    # -----------------------------------------------------
    # CLOSE WRITER
    # -----------------------------------------------------
    def close(self):
        self.writer.close()
        print("[Monitor] TensorBoard writer closed.")


In [ ]:
# =========================================================
# Revised Training Code with Monitoring & Control Included
# =========================================================

PHYSIONET_DIR = r"C:\Users\drsas\Project4\Rawdata\Physionet"
LEAD_INDEX = 0

CHECKPOINT_DIR = r"C:\Users\drsas\Project4\CheckPoints"
LOG_DIR = r"C:\Users\drsas\Project4\Logs"
TB_LOG_DIR = os.path.join(LOG_DIR, "tensorboard")
FINAL_MODEL_PATH = os.path.join(
    CHECKPOINT_DIR,
    "ECG_Generator_PhysioNet_Final.h5"
)

VIS_DIR = r"C:\Users\drsas\Project4\final_visualizations"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(TB_LOG_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

# =========================================================
# LOGGING SETUP
# =========================================================
log_file = os.path.join(
    LOG_DIR,
    f"training_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
)

logging.basicConfig(
    filename=log_file,
    filemode="w",
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

logging.info("WGAN-GP Training Started")

# CSV logging
csv_log_path = os.path.join(LOG_DIR, "training_metrics.csv")
csv_file = open(csv_log_path, mode="w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    "epoch",
    "critic_loss",
    "generator_loss",
    "epoch_time_sec",
    "checkpoint_saved"
])

tb_writer = tf.summary.create_file_writer(TB_LOG_DIR)

# =========================================================
# MODULE 7: MONITOR INITIALIZATION
# =========================================================
monitor = WGANMonitor(
    log_dir=TB_LOG_DIR,
    patience=10
)

# =========================================================
# LOAD ECG DATA (MODULE 2)
# =========================================================
print("\n[INFO] Loading PhysioNet ECG data...")
logging.info("Loading PhysioNet ECG data")

ecg_array = load_physionet_ecg(
    physionet_dir=PHYSIONET_DIR,
    seq_len=SEQ_LEN,
    lead_index=LEAD_INDEX
)

dataset = get_tf_dataset(ecg_array, BATCH_SIZE)

print("[INFO] ECG data ready")
logging.info("ECG data loaded successfully")

# =========================================================
# BUILD MODELS
# =========================================================
print("\n[INFO] Building Generator and Critic...")
generator = build_generator(SEQ_LEN, LATENT_DIM)
critic = build_critic(SEQ_LEN)

# =========================================================
# OPTIMIZERS
# =========================================================
g_optimizer = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.9)
c_optimizer = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.9)

# =========================================================
# CHECKPOINTING
# =========================================================
ckpt = tf.train.Checkpoint(
    generator=generator,
    critic=critic,
    g_optimizer=g_optimizer,
    c_optimizer=c_optimizer
)

ckpt_manager = tf.train.CheckpointManager(
    ckpt,
    CHECKPOINT_DIR,
    max_to_keep=5
)

# =========================================================
# GRADIENT PENALTY
# =========================================================
def gradient_penalty(real, fake):
    alpha = tf.random.uniform([real.shape[0], 1, 1], 0.0, 1.0)
    interpolated = alpha * real + (1.0 - alpha) * fake

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = critic(interpolated, training=True)

    grads = tape.gradient(pred, interpolated)
    grad_norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2]))

    return tf.reduce_mean((grad_norm - 1.0) ** 2)

# =========================================================
# TRAINING STEPS
# =========================================================
@tf.function
def train_critic(real_ecg):
    z = tf.random.normal((BATCH_SIZE, LATENT_DIM))

    with tf.GradientTape() as tape:
        fake_ecg = generator(z, training=True)
        c_real = critic(real_ecg, training=True)
        c_fake = critic(fake_ecg, training=True)
        gp = gradient_penalty(real_ecg, fake_ecg)

        loss = (
            tf.reduce_mean(c_fake)
            - tf.reduce_mean(c_real)
            + GP_LAMBDA * gp
        )

    grads = tape.gradient(loss, critic.trainable_variables)
    c_optimizer.apply_gradients(zip(grads, critic.trainable_variables))
    return loss


@tf.function
def train_generator():
    z = tf.random.normal((BATCH_SIZE, LATENT_DIM))

    with tf.GradientTape() as tape:
        fake_ecg = generator(z, training=True)
        loss = -tf.reduce_mean(critic(fake_ecg, training=True))

    grads = tape.gradient(loss, generator.trainable_variables)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_variables))
    return loss

# =========================================================
# TRAINING LOOP (WITH MONITORING & CONTROL)
# =========================================================

print("\n[INFO] Starting WGAN-GP training...\n")

epoch_c_loss_history = []
epoch_g_loss_history = []

for epoch in range(EPOCHS):

    start_time = time.time()

    g_losses = []
    c_losses = []

    for real_batch in dataset:
        for _ in range(N_CRITIC):
            c_losses.append(train_critic(real_batch))

        g_losses.append(train_generator())

    epoch_time = time.time() - start_time

    mean_c_loss = float(tf.reduce_mean(c_losses))
    mean_g_loss = float(tf.reduce_mean(g_losses))

    epoch_c_loss_history.append(mean_c_loss)
    epoch_g_loss_history.append(mean_g_loss)

    print(
        f"Epoch [{epoch+1:03d}/{EPOCHS}] | "
        f"C Loss: {mean_c_loss:.4f} | "
        f"G Loss: {mean_g_loss:.4f} | "
        f"Time: {epoch_time:.2f}s"
    )

    # ---------------- MONITOR ----------------
    monitor.log(epoch + 1, mean_g_loss, mean_c_loss)

    # NaN Detection
    if monitor.check_nan(mean_g_loss, mean_c_loss):
        print("[INFO] Training stopped due to NaN.")
        break

    # Early Stopping
    if monitor.early_stopping(mean_g_loss):
        print("[INFO] Early stopping activated.")
        break

    # ---------------- EXISTING CHECKPOINT ----------------
    checkpoint_saved = "NO"
    if (epoch + 1) % 5 == 0:
        ckpt_manager.save()
        checkpoint_saved = "YES"

    csv_writer.writerow([
        epoch + 1,
        mean_c_loss,
        mean_g_loss,
        epoch_time,
        checkpoint_saved
    ])

    with tb_writer.as_default():
        tf.summary.scalar("Critic_Loss", mean_c_loss, step=epoch + 1)
        tf.summary.scalar("Generator_Loss", mean_g_loss, step=epoch + 1)

print("\n[INFO] Training completed")
monitor.close()

# =========================================================
# FINAL VISUALISATION (UNCHANGED)
# =========================================================

plt.figure(figsize=(8, 5))
plt.plot(epoch_c_loss_history, label="Critic Loss")
plt.plot(epoch_g_loss_history, label="Generator Loss")
plt.legend()
plt.title("WGAN-GP Training Loss Curves")
plt.show()

real_sample = ecg_array[0]
z = tf.random.normal((1, LATENT_DIM))
fake_sample = generator(z, training=False)[0].numpy()

plt.figure(figsize=(10, 4))
plt.plot(real_sample.squeeze(), label="Real ECG")
plt.plot(fake_sample.squeeze(), label="Generated ECG")
plt.legend()
plt.title("Final Real vs Generated ECG")
plt.show()

generator.save(FINAL_MODEL_PATH)
print(f"[INFO] Final Generator saved to: {FINAL_MODEL_PATH}")

csv_file.close()
logging.info("Training completed and logs closed")


[Monitor] Logging to: C:\Users\drsas\Project4\Logs/tensorboard

[INFO] Loading PhysioNet ECG data...


FileNotFoundError: PhysioNet directory not found: C:\Users\drsas\Project4\Rawdata\Physionet

In [ ]:
# =========================================================
# Module 8: Inference API + Inference Engine
# =========================================================
# =========================================================
# ECG Inference Class
# =========================================================
class ECGInference:

    def __init__(self, model_path, latent_dim=100):
        """
        Initialize inference engine
        """
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model not found at: {model_path}")

        print(f"[Inference] Loading model from: {model_path}")
        self.generator = tf.keras.models.load_model(model_path)
        self.latent_dim = latent_dim

        print("[Inference] Model loaded successfully")

    # -----------------------------------------------------
    # Generate ECG Samples
    # -----------------------------------------------------
    def generate(self, n_samples=1):
        z = tf.random.normal((n_samples, self.latent_dim))
        generated = self.generator(z, training=False).numpy()

        print(f"[Inference] Generated ECG shape: {generated.shape}")
        return generated

    # -----------------------------------------------------
    # Visualize ECG Signal
    # -----------------------------------------------------
    def visualize(self, ecg_array, index=0):
        if index >= len(ecg_array):
            raise ValueError("Index exceeds number of samples")

        plt.figure(figsize=(10, 4))
        plt.plot(ecg_array[index].squeeze())
        plt.title("Generated ECG Signal")
        plt.xlabel("Time")
        plt.ylabel("Amplitude")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    # -----------------------------------------------------
    # Save ECG Signals
    # -----------------------------------------------------
    def save(self, ecg_array, save_dir="generated_ecg", format="npy"):
        os.makedirs(save_dir, exist_ok=True)

        if format == "npy":
            path = os.path.join(save_dir, "generated_ecg.npy")
            np.save(path, ecg_array)

        elif format == "csv":
            for i, sample in enumerate(ecg_array):
                path = os.path.join(save_dir, f"ecg_{i}.csv")
                np.savetxt(path, sample.squeeze(), delimiter=",")

        else:
            raise ValueError("Format must be 'npy' or 'csv'")

        print(f"[Inference] ECG saved to: {save_dir}")

    # -----------------------------------------------------
    # Full Pipeline
    # -----------------------------------------------------
    def run(self, n_samples=1, save=False, save_dir="generated_ecg"):
        ecg = self.generate(n_samples)
        self.visualize(ecg, index=0)

        if save:
            self.save(ecg, save_dir=save_dir)

        return ecg


# =========================================================
# INFERENCE ENGINE IMPLEMENTATION
# =========================================================
if __name__ == "__main__":

    # ---------------- CONFIGURATION ----------------
    MODEL_PATH = r"C:\Users\drsas\Project4\CheckPoints\ECG_Generator_PhysioNet_Final.h5"
    LATENT_DIM = 100
    N_SAMPLES = 5
    SAVE_OUTPUT = True
    SAVE_DIR = r"C:\Users\drsas\Project4\GeneratedECG"

    print("\n[INFO] Starting Inference Engine...\n")

    # ---------------- INITIALIZE ENGINE ----------------
    engine = ECGInference(
        model_path=MODEL_PATH,
        latent_dim=LATENT_DIM
    )

    # ---------------- RUN INFERENCE ----------------
    generated_ecg = engine.run(
        n_samples=N_SAMPLES,
        save=SAVE_OUTPUT,
        save_dir=SAVE_DIR
    )

    print("\n[INFO] Inference Completed Successfully")



In [ ]:
# =========================================================
# WGAN-Based Synthetic ECG Generator Deployment
# =========================================================

import os
import streamlit as st
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# =========================================================
# CONFIGURATION
# =========================================================
MODEL_PATH = r"C:\Users\drsas\Project4\CheckPoints\ECG_Generator_PhysioNet_Final.h5"
LATENT_DIM = 100
SEQ_LEN = 1248

# =========================================================
# PAGE SETTINGS
# =========================================================
st.set_page_config(
    page_title="Synthetic ECG Generator",
    page_icon="🫀",
    layout="wide"
)

st.title("🫀 WGAN-Based Synthetic ECG Generator")
st.write("Generate realistic synthetic ECG signals using trained WGAN-GP model.")

# =========================================================
# LOAD MODEL SAFELY
# =========================================================
@st.cache_resource
def load_generator(model_path):
    if not os.path.exists(model_path):
        return None
    return tf.keras.models.load_model(model_path)

generator = load_generator(MODEL_PATH)

if generator is None:
    st.error(f"❌ Model file not found at:\n{MODEL_PATH}")
    st.stop()

st.success("✅ Generator model loaded successfully")

# =========================================================
# SIDEBAR CONTROLS
# =========================================================
st.sidebar.header("Generation Settings")

n_samples = st.sidebar.slider(
    "Number of ECG Samples",
    min_value=1,
    max_value=20,
    value=1
)

generate_button = st.sidebar.button("Generate ECG")

# =========================================================
# GENERATION LOGIC
# =========================================================
if generate_button:

    st.info("Generating ECG...")

    z = tf.random.normal((n_samples, LATENT_DIM))
    generated_ecg = generator(z, training=False).numpy()

    st.success("Generation complete!")

    # -----------------------------------------------------
    # DISPLAY ECG SIGNALS
    # -----------------------------------------------------
    for i in range(n_samples):

        st.subheader(f"Generated ECG Sample {i+1}")

        fig, ax = plt.subplots(figsize=(10, 3))
        ax.plot(generated_ecg[i].squeeze())
        ax.set_title("Synthetic ECG Signal")
        ax.set_xlabel("Time")
        ax.set_ylabel("Amplitude")
        ax.grid(True)

        st.pyplot(fig)

    # -----------------------------------------------------
    # DOWNLOAD OPTION
    # -----------------------------------------------------
    st.download_button(
        label="Download Generated ECG (.npy)",
        data=np.array(generated_ecg).tobytes(),
        file_name="generated_ecg.npy",
        mime="application/octet-stream"
    )

# =========================================================
# FOOTER
# =========================================================
st.markdown("---")
st.markdown("Developed using WGAN-GP for Synthetic ECG Generation")


In [ ]:
# =========================================================
# ECG Classifier Using Real + GAN Generated Data
# =========================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# =========================================================
# CONFIGURATION (MUST MATCH GAN TRAINING)
# =========================================================
SEQ_LEN = 1248
LATENT_DIM = 100
N_CLASSES = 2      # 0 = Real, 1 = Generated
BATCH_SIZE = 32
EPOCHS = 10

PHYSIONET_DIR = r"C:\Users\drsas\Project4\Rawdata\Physionet"
GENERATED_DATA_PATH = r"C:\Users\drsas\Project4\GeneratedECG\generated_ecg.npy"

CHECKPOINT_DIR = r"C:\Users\drsas\Project4\CheckPoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# =========================================================
# BUILD CLASSIFIER
# =========================================================
def build_ecg_classifier(seq_len, n_classes):

    ecg_input = layers.Input(shape=(seq_len, 1), name="ecg_input")

    x = ecg_input

    for filters in [32, 64, 128]:
        x = layers.Conv1D(filters, 5, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.MaxPooling1D(2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    output = layers.Dense(n_classes, activation="softmax")(x)

    return Model(ecg_input, output, name="ECG_Classifier")

# =========================================================
# LOAD REAL ECG DATA (Module 2)
# =========================================================
print("\n[INFO] Loading real ECG from PhysioNet...")

X_real = load_physionet_ecg(
    physionet_dir=PHYSIONET_DIR,
    seq_len=SEQ_LEN,
    lead_index=0
)

print(f"[INFO] Real ECG shape: {X_real.shape}")

# =========================================================
# LOAD GENERATED ECG DATA (Module 8 Output)
# =========================================================
print("[INFO] Loading generated ECG data...")

if not os.path.exists(GENERATED_DATA_PATH):
    raise FileNotFoundError(
        f"Generated ECG file not found at: {GENERATED_DATA_PATH}"
    )

X_fake = np.load(GENERATED_DATA_PATH)

if len(X_fake.shape) == 2:
    X_fake = X_fake[..., np.newaxis]

print(f"[INFO] Generated ECG shape: {X_fake.shape}")

# =========================================================
# CREATE LABELS
# =========================================================
y_real = np.zeros(len(X_real))     # Label 0
y_fake = np.ones(len(X_fake))      # Label 1

# Combine
X = np.concatenate([X_real, X_fake], axis=0)
y = np.concatenate([y_real, y_fake], axis=0)

print(f"[INFO] Combined dataset shape: {X.shape}")

# =========================================================
# SPLIT DATA
# =========================================================
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42
)

print(f"[INFO] Training samples: {len(X_train)}")
print(f"[INFO] Validation samples: {len(X_val)}")
print(f"[INFO] Test samples: {len(X_test)}")

# =========================================================
# BUILD & COMPILE MODEL
# =========================================================
classifier = build_ecg_classifier(SEQ_LEN, N_CLASSES)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

classifier.summary()

# =========================================================
# TRAIN
# =========================================================
history = classifier.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# =========================================================
# SAVE MODEL
# =========================================================
MODEL_PATH = os.path.join(CHECKPOINT_DIR, "ECG_Classifier_GAN.h5")
classifier.save(MODEL_PATH)

print(f"[INFO] Classifier saved at: {MODEL_PATH}")

# =========================================================
# EVALUATION
# =========================================================
print("\n[INFO] Evaluating classifier...\n")

y_pred_prob = classifier.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print("\nClassification Report\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix\n")
print(confusion_matrix(y_test, y_pred))
